# Эксперименты с энкодерами для сервиса матчинга описания вакансий и резюме

## Описание проекта

**Цель проекта:** разработка и реализация программного решения для сопоставления текстовых описаний вакансии и резюме.

**Описание проекта:** разработка сервиса для платформы по поиску работы и сотрудников, позволяющего измерить релевантность резюме соискателя и вакансии. Задача приложения - выдавать численную оценку, насколько конкретный соискатель отвечает требованиям вакансии.

**Потенциальные варианты использования сервиса на платформе:** улучшение поиска вакансий соискателями, рекомендация вакансий, фильтрация нерелевантных кандидатов hr-специалистами, что в итоге повысит удовлетворенность всех пользователей продукта.

## Библиотеки и настроки

In [1]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=31791888a9ca052a1a87a1816956a93797172be7572f593c9418c2d6189896cd
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import torch

from transformers.pipelines import pipeline
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer, CrossEncoder, LoggingHandler, losses, InputExample, util


In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [5]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [6]:
# сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)

# сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

# сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

## Датасеты

### Датасет резюме HeadHunter

In [7]:
df_CV = pd.read_csv('/content/gdrive/MyDrive/datasets/df_CV.csv', on_bad_lines='skip', encoding='utf-8', sep=',')

In [8]:
df_CV.drop(['Пол, возраст', 'Последнее/нынешнее место работы', 'Последняя/нынешняя должность', 'Обновление резюме', 'Авто', 'Образование и ВУЗ'], axis=1, inplace=True)

In [9]:
df_CV = df_CV[['Ищет работу на должность:', 'ЗП', 'Опыт работы', 'Занятость', 'График', 'Город, переезд, командировки']]

In [10]:
df_CV.head(1)

,Ищет работу на должность:,ЗП,Опыт работы,Занятость,График,"Город, переезд, командировки"
0,Системный администратор,29000 руб.,"Опыт работы 16 лет 10 месяцев Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ ""СОШ № 1 г.Немана"" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации. Август 2002 — Август 2010 8 лет 1 месяц ТС ""ВЕСТЕР-ИНФО"" Старший продавец, директор отдела Продажи компьютерной техники","частичная занятость, проектная работа, полная занятость","гибкий график, полный день, сменный график, вахтовый метод, удаленная работа","Советск (Калининградская область) , не готов к переезду , не готов к командировкам"


In [11]:
df_CV.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44574 entries, 0 to 44573
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Ищет работу на должность:     44574 non-null  object
 1   ЗП                            44574 non-null  object
 2   Опыт работы                   44574 non-null  object
 3   Занятость                     44574 non-null  object
 4   График                        44574 non-null  object
 5   Город, переезд, командировки  44574 non-null  object
dtypes: object(6)
memory usage: 2.0+ MB


### Датасет вакансий HeadHunter

In [12]:
df_vacancies = pd.read_csv('/content/gdrive/MyDrive/datasets/df_vacancies.csv', on_bad_lines='skip', encoding='utf-8', sep=',')

In [13]:
df_vacancies = df_vacancies[df_vacancies['type'] == 'close']

In [14]:
df_vacancies.drop(['№', 'id', 'company', 'date_of_post', 'type', 'description'], axis=1, inplace=True)

In [15]:
df_vacancies.head(1)

,title,salary,experience,job_type,key_skills,location
0,Начальник участка,от 130000 RUR,От 1 года до 3 лет,"Полная занятость,полный день","AutoCAD,MS Excel,MS Word,Управление производственным персоналом,Составление графиков","Москва, улица Ивана Бабушкина, 4"


In [16]:
df_vacancies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109929 entries, 0 to 109928
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   title       109929 non-null  object
 1   salary      109929 non-null  object
 2   experience  109929 non-null  object
 3   job_type    109929 non-null  object
 4   key_skills  109929 non-null  object
 5   location    109929 non-null  object
dtypes: object(6)
memory usage: 5.9+ MB


## Эксперименты с энкодерами для задачи Semantic search

### multilingual-e5-large

In [85]:
model_name = 'intfloat/multilingual-e5-large'

In [86]:
retrieval_model = SentenceTransformer(model_name)

Сформируем эмбеддинги из датасета вакансий на части выборки для тестирования качества модели:

In [19]:
df_vacancies_concat = df_vacancies.apply(lambda row: '. '.join(row.astype(str)), axis=1)

In [20]:
vacancies_embedding_e5 = retrieval_model.encode(df_vacancies_concat, convert_to_tensor=True)

Сформируем из резюме запрос и определим функцию для семантического поиска подходящих вакансий по базе :

In [76]:
query = df_CV.iloc[[0]].apply(lambda row: '. '.join(row.astype(str)), axis=1).values[0]

In [22]:
query

'Системный администратор. 29000 руб.. Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники. частичная занятость, проектная работа, полная занятость. гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. Советск (Калининградская область) , не готов к переезду , не готов к командировкам'

In [78]:
def sem_search_results(model, query, corpus, corpus_embedding, top_k):
  ##### Sematic Search (Retrieve) #####
  question_embedding = model.encode(query, convert_to_tensor=True)
  hits = util.semantic_search(question_embedding, corpus_embedding, top_k=top_k)
  hits = hits[0]  # Get the hits

  # score all retrieved passages
  sem_search_res = [[query, corpus[hit['corpus_id']]] for hit in hits]
  sem_search_emb = [question_embedding[0]] + [corpus_embedding[hit['corpus_id']] for hit in hits]

  return sem_search_res, sem_search_emb, hits

Получим топ подходящих вакансий:

In [87]:
sem_search_res, sem_search_emb, hits_retrieved = sem_search_results(retrieval_model, query, df_vacancies_concat, vacancies_embedding_e5, 5)

In [25]:
sem_search_res[:5]

[['Системный администратор. 29000 руб.. Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники. частичная занятость, проектная работа, полная занятость. гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. Советск (Калининградская область) , не готов к переезду , не готов к командировкам',
  'Системный администратор. от 90000 до 140000 RUR. Более 6 лет. Полная занятость,полный день. Администрирование сетевого оборудования,Настройка сетевых подключений,Информационные технологии,Linux,Networking,Маршрутизация статическая,Маршрутизация динамическая,Телекоммуникации,Сетевые технологии,Высокий уровень самоорганизации,Умею начинать дела и дово

### rubert-tiny2

In [93]:
model_name = 'cointegrated/rubert-tiny2'

In [95]:
retrieval_model = SentenceTransformer(model_name)

In [28]:
vacancies_embedding_rubert_tiny2 = retrieval_model.encode(df_vacancies_concat, convert_to_tensor=True)

In [96]:
sem_search_res, sem_search_emb, hits_retrieved = sem_search_results(retrieval_model, query, df_vacancies_concat, vacancies_embedding_rubert_tiny2, 5)

In [97]:
sem_search_res[:5]

[['Системный администратор. 29000 руб.. Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники. частичная занятость, проектная работа, полная занятость. гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. Советск (Калининградская область) , не готов к переезду , не готов к командировкам',
  'Cистемный администратор / Специалист технической поддержки. от 50000 RUR. От 3 до 6 лет. Полная занятость,полный день. Настройка DNS,Active Directory,Администрирование серверов Windows,Администрирование,Кассовые операции,Подключение торгового оборудования к 1С,DHCP,КриптоПро,Опытный пользователь ПК,Настройка ПК,Администрирование сетевого оборудовани

### MiniLM-L12-v2

In [31]:
model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

In [80]:
retrieval_model = SentenceTransformer(model_name)

In [33]:
vacancies_embedding_MiniLML12v2 = retrieval_model.encode(df_vacancies_concat, convert_to_tensor=True)

In [34]:
sem_search_res, sem_search_emb, hits_retrieved = sem_search_results(retrieval_model, query, df_vacancies_concat, vacancies_embedding_MiniLML12v2, 5)

In [35]:
sem_search_res[:5]

[['Системный администратор. 29000 руб.. Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники. частичная занятость, проектная работа, полная занятость. гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. Советск (Калининградская область) , не готов к переезду , не готов к командировкам',
  'Системный администратор. от 40000 до 50000 RUR. От 1 года до 3 лет. Полная занятость,гибкий график. Настройка сетевых подключений,Ремонт ПК,Настройка ПО,Настройка ПК,Настройка серверов,Windows 7,Windows Xp,Выездное обслуживание,Многозадачность,Автоматизированное рабочее место (АРМ),Уверенный пользователь ПК. Саранск, Энергетическая улица, 6'],
 ['Си

### DeepPavlov rubert-base-cased-sentence

In [89]:
model_name = 'DeepPavlov/rubert-base-cased-sentence'

In [90]:
retrieval_model = SentenceTransformer(model_name)

In [38]:
vacancies_embedding_DeepPavlovRubertBaseCasedSentence = retrieval_model.encode(df_vacancies_concat, convert_to_tensor=True)

In [91]:
sem_search_res, sem_search_emb, hits_retrieved = sem_search_results(retrieval_model, query, df_vacancies_concat, vacancies_embedding_DeepPavlovRubertBaseCasedSentence, 5)

In [40]:
sem_search_res[:5]

[['Системный администратор. 29000 руб.. Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администратор Обслуживание ПК,установка ПО, ремонт, периферийной техники, Интернет локальная сеть. Ведение Электронного журнала, сайта организации.  Август 2002 — Август  2010 8 лет 1 месяц ТС "ВЕСТЕР-ИНФО" Старший продавец, директор отдела Продажи компьютерной техники. частичная занятость, проектная работа, полная занятость. гибкий график, полный день, сменный график, вахтовый метод, удаленная работа. Советск (Калининградская область) , не готов к переезду , не готов к командировкам',
  'Ученик на мебельное производство/сотрудник с опытом работы. от 30000 до 50000 RUR. Нет опыта. Стажировка,полный день. Пользователь ПК,Работа в команде. Кострома, Дровяная улица, 49'],
 ['Системный администратор. 29000 руб.. Опыт работы 16 лет 10 месяцев  Август 2010 — по настоящее время 8 лет 10 месяцев МАОУ "СОШ № 1 г.Немана" Системный администрато

## Выводы

Выводы по результатам экспериментов: модель multilingual-e5-large показала наилучшие результаты по качеству получаемых эмбеддингов - в топ релевантных вакансий семантического поиска попадают результаты с идентичной должностью. Кроме того, есть совпадения по региону и преимущественная близость мэтчей в опыте работы.